### Importing and Initial Preping of Data

In [17]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [18]:
sc=StandardScaler()

In [19]:
bf=data = pd.read_csv('BlackFridaySalesResized.csv')

In [20]:
bf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159999 entries, 0 to 159998
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   User_ID                     159999 non-null  int64  
 1   Product_ID                  159999 non-null  object 
 2   Gender                      159999 non-null  object 
 3   Age                         159999 non-null  object 
 4   Occupation                  159999 non-null  int64  
 5   City_Category               159999 non-null  object 
 6   Stay_In_Current_City_Years  159999 non-null  object 
 7   Marital_Status              159999 non-null  int64  
 8   Product_Category_1          159999 non-null  int64  
 9   Product_Category_2          110328 non-null  float64
 10  Product_Category_3          48940 non-null   float64
 11  Purchase                    159999 non-null  int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 14.6+ MB


In [21]:
bf.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [22]:
missing_values = bf.isnull().sum().sort_values(ascending = False)
missing_values = missing_values[missing_values > 0]/bf.shape[0]
print(f'{missing_values *100} %')

Product_Category_3    69.412309
Product_Category_2    31.044569
dtype: float64 %


In [23]:
bf.fillna(0,inplace=True)

In [24]:
bf['Age']=bf['Age'].apply(lambda x: 1 if x=='0-17' else( 2 if x=='18-25' else(3 if x in ('26-35','36-45') else (4 if x in ('46-50','51-55','55+') else x ) )))

In [25]:
bf=pd.get_dummies(data=bf,columns=['Gender','City_Category'],drop_first=True)

In [26]:
bf["Stay_In_Current_City_Years"]=bf["Stay_In_Current_City_Years"].apply(lambda x: str(x).replace("+","") if "+" in x else x)
bf["Stay_In_Current_City_Years"]=bf["Stay_In_Current_City_Years"].apply(lambda x: int(x))

In [27]:
bf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159999 entries, 0 to 159998
Data columns (total 13 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   User_ID                     159999 non-null  int64  
 1   Product_ID                  159999 non-null  object 
 2   Age                         159999 non-null  int64  
 3   Occupation                  159999 non-null  int64  
 4   Stay_In_Current_City_Years  159999 non-null  int64  
 5   Marital_Status              159999 non-null  int64  
 6   Product_Category_1          159999 non-null  int64  
 7   Product_Category_2          159999 non-null  float64
 8   Product_Category_3          159999 non-null  float64
 9   Purchase                    159999 non-null  int64  
 10  Gender_M                    159999 non-null  uint8  
 11  City_Category_B             159999 non-null  uint8  
 12  City_Category_C             159999 non-null  uint8  
dtypes: float64(2),

In [28]:
#dropping columns which have all the unique values
bf.drop(columns=['User_ID','Product_ID'],inplace=True)

In [30]:
bf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159999 entries, 0 to 159998
Data columns (total 11 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Age                         159999 non-null  int64  
 1   Occupation                  159999 non-null  int64  
 2   Stay_In_Current_City_Years  159999 non-null  int64  
 3   Marital_Status              159999 non-null  int64  
 4   Product_Category_1          159999 non-null  int64  
 5   Product_Category_2          159999 non-null  float64
 6   Product_Category_3          159999 non-null  float64
 7   Purchase                    159999 non-null  int64  
 8   Gender_M                    159999 non-null  uint8  
 9   City_Category_B             159999 non-null  uint8  
 10  City_Category_C             159999 non-null  uint8  
dtypes: float64(2), int64(6), uint8(3)
memory usage: 10.2 MB
